In [1]:
from ethpandaops_python.preprocessor import Preprocessor
from holoviews import opts
import nest_asyncio
import polars as pl
import panel as pn


nest_asyncio.apply()
pn.extension("plotly", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Data Query

In [2]:
# labeled blobs - https://dune.com/queries/3521610
arbitrum: str = "0xC1b634853Cb333D3aD8663715b08f41A3Aec47cc"
linea: str = "0xa9268341831eFa4937537bc3e9EB36DbecE83C7e"
base: str = "0x5050F69a9786F081509234F1a7F4684b5E5b76C9"
starknet: str = "0x2C169DFe5fBbA12957Bdd0Ba47d9CEDbFE260CA7"

num_days: int = 1

In [3]:
preprocessor: Preprocessor = Preprocessor(
    blob_producer=base,
    period=num_days,
    network="mainnet",  # mainnet
)

In [5]:
slot_count_breakdown_df: pl.DataFrame = preprocessor.create_slot_count_breakdown_df()

slot_inclusion_df: pl.DataFrame = (
    preprocessor.create_slot_inclusion_df()
    # there is an outlier in the data that has an unusually high slot count, I think it might be an error with the xatu mempool data.
    # additionally there is currently a single tx with 5 blobs and 1 with 7. Since it's a single row, I think it's easier to just discard and ignore for now
    .filter(pl.col("slot inclusion rate") < 250)
    .filter(pl.col("slot inclusion rate (50 blob average)") < 40)
)

slot_gas_bidding_df: pl.DataFrame = preprocessor.create_slot_gas_bidding_df()

In [6]:
# Histogram count breakdown
slot_chance_bar_chart = slot_count_breakdown_df.plot.barh(
    ylabel="number of txs",
    title="Slot Inclusion Breakdown",
    stacked=True,
    width=1000,
    height=400,
)

### Slot Inclusion Time Charts

In [8]:
slot_inclusion_df.columns

['versioned_hash',
 'nonce',
 'event_date_time_min',
 'event_date_time_max',
 'blob_hashes_length',
 'blob_sidecars_size',
 'fill_percentage',
 'blob_gas',
 'blob_gas_fee_cap',
 'gas_price',
 'gas_tip_cap',
 'gas_fee_cap',
 'hash',
 'from',
 'to',
 'submission_count',
 'slot',
 'slot_time',
 'block_root',
 'kzg_commitment',
 'meta_network_name',
 'blob_size',
 'blob_empty_size',
 'beacon_inclusion_time',
 'slot_inclusion_rate',
 'slot_inclusion_rate_50_blob_avg',
 '2_slot_target_inclusion_rate']

In [10]:
slot_inclusion_line_chart = slot_inclusion_df.select(
    "slot_time",
    "slot_inclusion_rate",
    "slot_inclusion_rate_50_blob_avg",
    "2_slot_target_inclusion_rate",
    "submission_count",
).plot.line(
    x="slot_time",
    y=[
        "slot_inclusion_rate",
        "slot_inclusion_rate_50_blob_avg",
        "2_slot_target_inclusion_rate",
    ],
    color=["blue", "red", "black"],
    ylabel="Beacon Block Inclusion (block)",
    xlabel="Slot Date Time",
    title="Historical Slot Inclusion",
    width=1000,
    height=400,
    shared_axes=True,
)

### Priority Fee Effectiveness for Faster Slot Inclusion

In [11]:
slot_gas_bidding_df.head(5)

block_number,max_priority_fee_per_gas_gwei,effective_gas_price_gwei,priority_fee_bid_percent_premium,slot_inclusion_rate,submission_count,min_block_gas_gwei,gas_fluctuation_gwei,gas_fluctuation_percent
i64,f64,f64,f64,f64,u32,f64,f64,f64
19634403,2,34.629,5.776,1,1,32.629,4.484999999999999,13.745441171963591
19634417,2,30.63,6.53,2,1,28.63,3.9989999999999988,13.967865874956336
19634431,2,27.435,7.29,2,1,25.435,3.1950000000000003,12.561431098879499
19634443,2,26.101,7.663,1,1,24.101,1.3339999999999996,5.535040039832371
19634457,2,25.692,7.785,2,1,23.692,0.4089999999999989,1.7263211210535157


In [14]:
# priority fee scatter plot
priority_fee_premium_chart = (
    slot_gas_bidding_df.group_by("slot_inclusion_rate")
    .agg(
        pl.col("gas_fluctuation_percent").mean(),
        pl.col("priority_fee_bid_percent_premium").mean(),
    )
    .sort(by="slot_inclusion_rate")
    .plot.scatter(
        x="slot_inclusion_rate",
        y="priority_fee_bid_percent_premium",
        ylabel="priority fee bid premium %",
        title="mean priority fee bid premium per slot inclusion rate",
        width=1000,
        height=400,
    )
)

In [16]:
volin_chart = (
    slot_gas_bidding_df.sort(by="slot_inclusion_rate")
    .drop_nulls()
    .plot.violin(
        y="priority_fee_bid_percent_premium",
        by="slot_inclusion_rate",
        c="slot_inclusion_rate",
        width=1000,
        height=400,
    )
)

line_chart_bid_premium = (
    slot_gas_bidding_df.group_by("slot_inclusion_rate")
    .agg(
        pl.col("priority_fee_bid_percent_premium").median(),
        pl.col("effective_gas_price_gwei").mean(),
    )
    .sort(by="slot_inclusion_rate")
    .drop_nulls()
    .rename(
        {
            "priority_fee_bid_percent_premium": "priority fee bid premium (%)",
            "effective_gas_price_gwei": "block gas price (gwei)",
        }
    )
    .plot.line(
        x="slot_inclusion_rate",
        y=["priority fee bid premium (%)", "block gas price (gwei)"],
        color=["g", "r"],
        ylabel="bid premium (%, gwei)",
        title="gas bid premium probability distributions",
    )
)

In [17]:
violin_line_gas_chart = volin_chart * line_chart_bid_premium

### Assemble Dashboard

In [18]:
dash = pn.Column(
    pn.pane.Markdown(
        """
        # EIP-4844 Slot Inclusion Dashboard

        ## About
        This dashboard shows detailed analytics for blobs and how fast they are included into the next slot as well as the efficiency of using EIP-1559 priority fees
        as a bidding mechanism for faster slot inclusion. As of April 11, 2024, the data is currently restricted to Base posting data over a 5 day period with 
        plans to open up to other rollups soon. This dashboard is made using [Xatu Data](https://github.com/ethpandaops/xatu-data?tab=readme-ov-file) for EL mempool and Beacon chain data and [Hypersync](https://github.com/enviodev/hypersync-client-python) 
        for transaction gas data for the [EIP-4844 data challenge](https://esp.ethereum.foundation/data-challenge-4844).

        ## Summary of Results
        - Blobs have a highly variable slot inclusion rate, with a nontrivial amount of blobs taking 3+ slots to be included.   
        - Higher EIP-1559 priority fees correlate with longer slot inclusion rates.
        - Uncertainty around block gas and priority fee bidding premiums increase with longer slot inclusion rates.
        
        ## Slot Inclusion Methodology
        When a transaction is resubmitted with updated gas parameters, the transaction hash changes. For example take this blob reference hash - 0x01c738cf37c911334c771f2295c060e5bd7d084f347e4334863336724934c59a. 
        On [etherscan](https://etherscan.io/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69) we can see that the transaction was replaced by the user. We can see on Ethernow that the transaction contains 
        the same blob reference hash in both the [original tx](https://www.ethernow.xyz/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69?batchIndex=1) and the [resubmitted tx](https://www.ethernow.xyz/tx/0x5a4094662bd05ff3639a8979927ab527e007a6925387951a9c1b3d2958b13a86?batchIndex=1).
        
        We can measure the total time that a blob hash sat in the mempool by subtracting the original tx was first seen from the slot time, when it eventually is finalized on the beacon chain. 
        In this particular example, the total time that the blob sat in the mempool was not from 18:56:27 to 18:57:11 (4 slots), but really 18:54:29 to 18:57:11 (14 slots)
            """
    ),
    pn.Column(
        pn.pane.Markdown(
            """
            ## Slot Inclusion Breakdown
            This bar chart represents the number of transactions that are being included within 1 slot, 2 slots, and 3+ slots. 
            * 1 slot = fastest possible inclusion time
            * 2 slots = good inclusion time
            * 3+ slots = slow inclusion time
            """
        ),
        slot_chance_bar_chart.opts(axiswise=True),
        pn.pane.Markdown(
            """
        ## Slot Inclusion Rates
        This time-series chartshows the slot inclusion performance over time from when it's first seen in the mempool to when it gets finalized in the beacon block.
        A 50 blob slot inclusion average is taken to smooth out the performance. The target slot inclusion rate is 2. To the right is the overall distribution of the slot inclusion rate.
        While a lot of blobs get finalized on the beacon chain within 1-2 blocks, there are still a non-trivial amount of outliers that can take upwards of 5-10 blocks before finalizing.
            """
        ),
        pn.Row(slot_inclusion_line_chart.opts(axiswise=True)),
        pn.pane.Markdown(
            """
        ## EIP-1559 Priority Fee Premium Correlation with Slot Rates
        The scatterplot shows the priority fee bid premium. There isn't a strong correlation between higher priority fee bid premiums and shorter slot inclusion rates. On the contrary,
        the longer it takes for the blob to be included, the priority fee bid premium starts to increase.
            """
        ),
        pn.Row(priority_fee_premium_chart),
        pn.pane.Markdown(
            """
        ## Slot Inclusion Bidding Unpredictability 
        This major takeaway is that uncertainty around both block gas and priority fee bidding tends to increase the longer the it takes for a blob to be included in a slot. There are three major 
        characteristics in the chart - one violin plot for the probability distribution and two trend lines for priority fee bids and block gas prices.
        * violin plots for each slot inclusion rate, showing the probability distribution of the priority gas bidding premium for each inclusion slot. 
        The higher the slot inclusion rate, the more uncertainty there is around 
        * the green line shows the median trend line for the priority fee bid premium (in %), which tends to drift higher as slot inclusion rate gets higher.
        * the red line shows the mean trend line for the block gas price (in gwei), which drifts upwards was slot inclusion rate gets higher.
            """
        ),
        pn.Row(violin_line_gas_chart),
    ),
)

In [20]:
# dash.show()
dash.servable()

Launching server at http://localhost:42269


Gtk-Message: 15:07:48.780: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:07:48.781: Failed to load module "canberra-gtk-module"


Opening in existing browser session.
